# CWT scalograms submission

Loading model, apply feature engineering to test eegs, submit predictions.

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import regularizers
# import matplotlib.pyplot as plt
import sys

# Flags for working on different hardware.
# flag_kaggle = True
# flag_FW = True
flag_LN = True

try:
    if flag_kaggle:
        # sys.path.insert(0, '../lib')
        base_dir = '/kaggle/input/hms-harmful-brain-activity-classification'
        devset_dir = '/kaggle/input/hms-cwt-scalograms-single-numpy-v1'
except:
    pass 

try:
    if flag_FW:
        sys.path.insert(0, '../lib')
        base_dir = '../../kaggle_data/hms'
        devset_dir = '../data'
except:
    pass 

try:
    if flag_LN:
        sys.path.insert(0, '../lib')
        base_dir = '../../data/hms'
        devset_dir = '../data'
except:
    pass 


from KLmetric import score


path_train = f'{devset_dir}/05_single_cwt_v1_train.npy'
path_train_items = f'{devset_dir}/05_single_cwt_v1_train_items.npy'
path_val = f'{devset_dir}/05_single_cwt_v1_val.npy'
path_val_items = f'{devset_dir}/05_single_cwt_v1_val_items.npy'
path_test = f'{devset_dir}/05_single_cwt_v1_test.npy'
path_test_items = f'{devset_dir}/05_single_cwt_v1_test_items.npy'

2024-03-10 12:53:32.184100: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Definitions


## Load pretrained models

In [5]:
# model = keras.model.load()

## Building a function for preprocessing

In [8]:
df_traincsv = pd.read_csv(f'{base_dir}/train.csv')
idxs_test = np.load(f'../data/03_stratified_v2_idxs_test.npy')
df_test = df_traincsv.loc[idxs_test]


In [9]:
test_items = np.load(path_test_items)

FileNotFoundError: [Errno 2] No such file or directory: '../data/05_single_cwt_v1_test_items.npy'

## Train

In [3]:
#
# Data generator using numpy and no pandas.
#
# scalograms
# 30 seconds slice (I think)
# 5 channels (LP, RP, LT, RP, C)
#

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=True):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        # self.n_freqs = 40

        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))
        y = np.empty((true_size, self.n_classes), dtype=float)

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]
            # Store solution
            y[i,:] = item[-6:]

        return X, y

In [4]:

def make_model(input_shape, num_classes):
    input_layer = keras.layers.Input(input_shape)

    #max1 = keras.layers.MaxPooling1D(pool_size=2)(input_layer)
    
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same")(input_layer)
    #conv1 = keras.layers.BatchNormalization()(conv1)
    # conv1 = keras.layers.MaxPooling2D(pool_size=8)(conv1)
    conv1 = keras.layers.ReLU()(conv1)
    
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=5, padding="same")(conv1)
    #conv2 = keras.layers.BatchNormalization()(conv2)
    # conv2 = keras.layers.MaxPooling2D(pool_size=8)(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv2D(filters=256, kernel_size=3, padding="same")(conv2)
    #conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.MaxPooling2D(pool_size=2)(conv3)
    conv3 = keras.layers.ReLU()(conv3)

#     conv4 = keras.layers.Conv2D(filters=512, kernel_size=3, padding="same")(conv3)
#     conv4 = keras.layers.BatchNormalization()(conv4)
#     conv4 = keras.layers.MaxPooling2D(pool_size=4)(conv4)
#     conv4 = keras.layers.ReLU()(conv4)

    fltn  = keras.layers.Flatten()(conv3) 
    
    relu1 = keras.layers.Dense(128)(fltn)
    relu1 = keras.layers.ReLU()(relu1)

#     relu2 = keras.layers.Dense(64)(relu1)
#     relu2 = keras.layers.ReLU(64)(relu2)

#     lin = keras.layers.Dense(2)(relu2)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(relu1)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


In [5]:
# Parameters
params = {
    'batch_size': 32,
    'n_classes': 6,
    'shuffle': True
    }

training_generator = DataGenerator(path_train_items, path_train , **params)
validation_generator = DataGenerator(path_val_items, path_val, **params)

print("Observations in training set:", training_generator.__len__()*params['batch_size'])
print("Observations in validation set:", validation_generator.__len__()*params['batch_size'])


Observations in training set: 12192
Observations in validation set: 2176


In [6]:
checkpoint_filepath = 'checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_kl_divergence',
    mode='min',
    save_best_only=True)


In [ ]:
# opt = keras.optimizers.SGD(
#     learning_rate=0.005,
#     momentum=0.00,
# )

opt = keras.optimizers.Adam(
    learning_rate=0.005,
)

dim = training_generator.get_dim()

model = make_model(input_shape=dim, num_classes=6)

# model.load_weights(checkpoint_filepath)

model.compile(optimizer=opt,
            loss=tf.keras.losses.KLDivergence(),
            metrics=[tf.keras.metrics.KLDivergence()])

model.fit(training_generator, epochs=5,
          validation_data=validation_generator,
          callbacks=[model_checkpoint_callback])

Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/381 ━━━━━━━━━━━━━━━━━━━━ 3:10:27 30s/step - kl_divergence: 1.2491 - loss: 1.2491

I0000 00:00:1710054553.910516     109 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710054553.927697     109 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - kl_divergence: 12.0517 - loss: 12.0520

W0000 00:00:1710054656.031345     108 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


381/381 ━━━━━━━━━━━━━━━━━━━━ 147s 307ms/step - kl_divergence: 12.0520 - loss: 12.0523 - val_kl_divergence: 13.1665 - val_loss: 13.1825
Epoch 2/5
  1/381 ━━━━━━━━━━━━━━━━━━━━ 6:06:20 58s/step - kl_divergence: 11.8154 - loss: 11.8154

In [11]:
model.save("model_cwt_057.keras")

## Score

In [9]:
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

#
# Test Data generator: for predicting
# using own test set.
# (Not for predicting LB)
#

class TestDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=False):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        # pass 
        
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]

        return X
    
                
params = {
    'batch_size': 32,
    'n_classes': 6,
    }

test_generator = TestDataGenerator(path_test_items, path_test, **params)

y_pred = model.predict(test_generator)

39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step


## Scoring without submission

Using a local test set.

In [10]:
test_items = np.load(path_test_items)
# test_items = np.load(f'{devset_dir}/03_single_spectrograms_reduced_v1_test_items.npy')
df_test_items = pd.DataFrame(test_items)
df_test_items[0] = df_test_items[0].astype(int)

sub = pd.DataFrame({'eeg_id':df_test_items[0]})
sub[TARGETS] = np.round(y_pred,6)
sub.to_csv('submission.csv',index=False)

df_test_scoring = df_test_items[[0,4,5,6,7,8,9]]
df_test_scoring.columns = sub.columns
# df_test_scoring

score(df_test_scoring, sub, 'eeg_id')

/tmp/ipykernel_1321/1435129968.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution[col] = solution[col].astype(float)
/tmp/ipykernel_1321/1435129968.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution[col] = solution[col].astype(float)
/tmp/ipykernel_1321/1435129968.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

0.5706095955828902

# Uniform probabilities classificator

In [13]:
# test_npy = np.load(path_test)
# test_items = np.load(path_test_items)

# y_pred = np.ones((test_items.shape[0],6),dtype=float)
# y_pred[:,0:4] = y_pred[:,0:4] * 0.167
# y_pred[:,4:] = y_pred[:,4:] * 0.166

# df_test_items = pd.DataFrame(test_items)
# df_test_items[0] = df_test_items[0].astype(int)

# sub = pd.DataFrame({'eeg_id':df_test_items[0]})
# sub[TARGETS] = np.round(y_pred,6)
# # sub.to_csv('submission.csv',index=False)

# df_test_scoring = df_test_items[[0,4,5,6,7,8,9]]
# df_test_scoring.columns = sub.columns

# score(df_test_scoring, sub, 'eeg_id')


/tmp/ipykernel_33/1435129968.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution[col] = solution[col].astype(float)
/tmp/ipykernel_33/1435129968.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solution[col] = solution[col].astype(float)
/tmp/ipykernel_33/1435129968.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

1.3718073504111403

# Submit to LB